<a href="https://colab.research.google.com/github/mohamedchebaane/coursera_test/blob/master/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3

In [2]:
import json

In [3]:
from datetime import datetime

In [4]:
timeframe = '2024-07'

In [9]:
sql_transaction = {}

In [10]:
connection = sqlite3.connect('{}.db'.format(timeframe))

In [11]:
c = connection.cursor()

In [14]:
def create_table():
    c.execute("CREATE TABLE IF NOT EXISTS parent_reply(parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT)""")

In [17]:
def format_data(data):
    data = data.replace("\n"," newlinechar ")

In [21]:
def find_existing_score(pid):
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print("find_parent", e)
        return False



In [23]:
def acceptable(data):
    if len(data.split(' ')) > 50 or len(data) < 1:
        return False
    elif len(data) > 1000:
        return False
    elif data =='[deleted]' or data == '[removed]':
        return False
    else:
        return True

In [24]:
def find_parent(pid):
    try:
      sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
      c.execute(sql)
      result = c.fetchone()
      if result != None:
          return result[0]
      else: return False
    except Exception as e:
      #print("find_parent", e)
      return False




In [ ]:
def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for a in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
        connection.commit()
        sql_transaction = []

In [ ]:
def sql_insert_replace_comment(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, subreddit = ?, unix = ?, score = ? WHERE parent_id =?;""".format(parentid, commentid, comment, subreddit, unix, score )
        transaction_bldr(sql)
    except Exception as e:
        print('s-UPDATE insertion',str(e))

def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parent_id, commentid, parent, comment, subreddit, unix, score)
        transaction_bldr(sql)
    except Exception as e:
        print('s-PARENT insertion',str(e))
def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});""".format(parent_id, commentid, comment, subreddit, unix, score)
        transaction_bldr(sql)
    except Exception as e:
        print('s-NO_PARENT insertion',str(e))

In [25]:
if __name__ == "__main__":
    create_table()
    row_counter = 0
    paired_rows = 0

    f = open('D:\\ml-25m\\movies.csv'):
    #with open("D:\\ml-25m\\movies.csv".format(timeframe.split('-')[0], timeframe), buffering=1000) as f:
        for row in f:
            row_counter += 1
            row = json.loads(row)
            parent_id = row['parent_id']
            body = format_data(row['body'])
            created_utc = row['created_utc']
            score = row['score']
            subreddit = row['subreddit']
            comment_id = row['name']
            parent_data = find_parent(parent_id)

            if score >= 2:
                if acceptabel(body):
                    existing_comment_score = find_existing_score(parent_id)
                    if existing_comment_score:
                        if score > existing_comment_score:
                            sql_insert_replace_comment(comment_id, parent_id, parent_data, body, subreddit, created_utc, score)
                    else:
                        if parent_data:
                            sql_insert_has_parent(comment_id, parent_id, parent_data, body, subreddit, created_utc, score)
                            paired_rows +=1
                        else:
                            sql_insert_no_parent(comment_id, parent_id, parent_data, body, subreddit, created_utc, score)


            if row_counter % 100000 == 0:
                print("Ttal rows read: {}, Paired rows: {}, Time: {}".format(row_counter, paired_rows, str(datetime.now())))


FileNotFoundError: [Errno 2] No such file or directory: 'D:/ml-25m/movies.csv'